In [10]:
#!pip -q install panel ipyleaflet ipywidgets_bokeh geopandas hvplot xarray_leaflet
import pandas as pd
import panel as pn
import holoviews as hv
import hvplot.pandas
import ipyleaflet
import geopandas as gpd
#import xarray_leaflet
from ipyleaflet import Map, CircleMarker, GeoJSON, LayerGroup, MarkerCluster
from holoviews.streams import Stream, param
import json

import xarray as xr
import panel as pn
import holoviews as hv
import ipyleaflet
import geopandas as gpd
from ipyleaflet import Map, GeoJSON
from holoviews.streams import Stream, param
import json
import folium
import folium.plugins
import matplotlib.pyplot as plt
# Initialize Panel extension
#pn.extension('plotly', 'ipywidgets')
#pn.extension('ipywidgets')

from leafmap import leafmap
import numpy as np

import html
import leafmap.foliumap as leafmap
import random

pn.extension("ipywidgets", "folium", sizing_mode="stretch_width")

In [2]:
huc2_gdf = gpd.read_file('../data/huc2.geojson')

huc2_gdf['huc2'] = huc2_gdf['huc2'].astype('int32') 
huc2_14 = huc2_gdf[huc2_gdf['huc2']==14] 

huc10_gdf = gpd.read_file('../data/huc10_clusters_loc.geojson')
huc10_gdf.geometry = huc10_gdf.simplify(tolerance=0.01, preserve_topology=True)

In [3]:
huc10_swe = pd.read_csv('../data/swe_totals_ucrb.csv') 
huc10_swe = huc10_swe.rename(columns={'swe_snv_sum_m3':'Calculated SWE', 'swe_reanalysis_sum_m3':'Reanalysis SWE'}) 

In [4]:
swe_reanalysis = xr.open_dataset('../data/SWE_reanalysis_32yrs.nc',mask_and_scale=True)['SWE_Post'].rio.clip_box(miny=36,maxy=42,minx=-110,maxx=-100).where(lambda x: x>0).compute() 
swe_calculated = xr.open_dataset('../data/swe_calculated_snv.nc',mask_and_scale=True)['SWE_Post'].rio.write_crs('EPSG:32611').rio.reproject_match(swe_reanalysis).compute()

In [5]:
center = [huc10_gdf.to_crs('EPSG:4326').unary_union.centroid.y,huc10_gdf.to_crs('EPSG:4326').unary_union.centroid.x]

In [11]:

# Create the ipyleaflet map
def create_ipyleaflet_map():
    
    m = Map(center=center, zoom=6, scroll_wheel_zoom=True)

    # Add HUC2 GeoJSON layer
    huc10_layer = GeoJSON(
        data=json.loads(huc10_gdf.to_crs('EPSG:4326').to_json()),
        style={'color': 'black', 'weight': 2, 'fillColor': 'blue', 'fillOpacity': 0.1},
        hover_style={'fillColor': 'red', 'fillOpacity': 0.2},
        name='HUC10' # check
    )

    huc2_layer = GeoJSON(
        data=json.loads(huc2_14.to_crs('EPSG:4326').to_json()),
        style={'color': 'black', 'weight': 3, 'fillColor': 'none'},
        name='HUC2' # check
    )
    
    # Add click event to GeoJSON layer
    def on_click(event, feature, **kwargs):
        properties = feature['properties']
        code = properties.get('huc10', 'Unknown')
        #click_stream.event(code=code)
        click_stream.code = code
        click_stream.param.trigger('code')
        center = [huc10_gdf[huc10_gdf['huc10'] == code].geometry.centroid.y.values[0], huc10_gdf[huc10_gdf['huc10'] == code].geometry.centroid.x.values[0]]
        update_folium_map()
        print(f'HUC code clicked: {code}')  # Debugging statement

    huc10_layer.on_click(on_click)
    m.add_layer(huc2_layer)
    m.add_layer(huc10_layer)

    return m

def create_folium_map(year,code=None):
    m = leafmap.Map(center=center,zoom=6)
    m.split_map(left_layer=swe_reanalysis.sel(Year=1990), left_args={'vmin':0,'vmax':1,'cmap':'Blues','nodata':np.nan}, right_layer=swe_calculated.sel(Year=0),right_args={'vmin':0,'vmax':1,'cmap':'Blues','nodata':np.nan})
    legend_dict = {
        'Left Layer': 'SWE Reanalysis',
        'Right Layer': 'SWE Calculated'
    }
    if code:
        m.add_gdf(huc10_gdf[huc10_gdf['huc10'] == code], layer_name='HUC10',zoom_to_layer=True, style={'color': 'black', 'weight': 2,'fillColor': 'blue', 'fillOpacity': 0.0},info_mode=None)
        #huc10_gdf[huc10_gdf['huc10'] == code].boundary.plot(color='black',linewidth=2)
        
    m.add_legend(title='Layers Legends', legend_dict=legend_dict)
    return m


# Create the map
ipyleaflet_map = create_ipyleaflet_map()
# Define a stream to capture map clicks
class ClickStream(Stream):
    code = param.String(default='1401000107')

# Instantiate the click stream
click_stream = ClickStream()

def create_plot(code):
    code = int(code)
    print(f'Code received in create_plot: {code}')  # Debugging statement
    filtered_data = huc10_swe[huc10_swe['huc10_id'] == code]
    if filtered_data.empty:
        return hv.Text(0.5, 0.5, f'No data for HUC Code: {code}', halign='center')
    plot = filtered_data.hvplot.line(x='year', y=['Calculated SWE', 'Reanalysis SWE'],
                                     value_label='Total SWE in Watershed ($m^3$)',
                                     title=f"{filtered_data.iloc[0]['name']}")
    plot.opts(legend_position='bottom', legend_cols=2)
    return plot

# Define a DynamicMap that updates based on the click stream
dynamic_map = hv.DynamicMap(lambda code: create_plot(code), streams=[click_stream.param.code])



# Convert ipyleaflet map to Panel

def update_folium_map(event=None):
    if event:
        year = event.new
    else:
        year = year_slider.value_throttled
    new_folium_map = create_folium_map(year,code=click_stream.code)
    new_escaped_html = html.escape(new_folium_map.to_html())
    new_iframe_html = f'<iframe srcdoc="{new_escaped_html}" style="height:100%; width:100%" frameborder="0"></iframe>'
    folium_pane.object = new_iframe_html

folium_map = create_folium_map(1990)  # Start with the initial year
#folium_pane = pn.pane.plot.HTML(html.escape(folium_map.to_html()), height=400, width=400)
escaped_html = html.escape(folium_map.to_html())

# Create iframe embedding the escaped HTML and display it
iframe_html = f'<iframe srcdoc="{escaped_html}" style="height:100%; width:100%" frameborder="0"></iframe>'

# Display iframe in a Panel HTML pane
folium_pane = pn.pane.HTML(iframe_html, height=350, sizing_mode="stretch_width")

year_slider = pn.widgets.IntSlider(name='Year', format='0', start=1990, end=2021)
year_slider.param.watch(update_folium_map, 'value_throttled')



ipyleaflet_pane = pn.pane.IPyWidget(ipyleaflet_map, height=400, width=400)

# Create the layout

layout = pn.Column(
    pn.pane.Markdown(
        """
        ## Interactive Snow Analysis Dashboard
        Explore snow water equivalent (SWE) data across different regions and years.
        """, margin=(0, 0, 20, 0)
    ),
    pn.Row(ipyleaflet_pane, folium_pane),
    pn.Row(year_slider,),
    pn.Row(pn.panel(dynamic_map))
)
# layout = pn.Row(ipyleaflet_pane, pn.panel(dynamic_map, height=400, width=400),folium_pane,year_slider)
# Create a custom template
template = pn.template.MaterialTemplate(
    title='Whoa! Is that Snow?',
    favicon='../snow_favicon.ico',  
)

# Add the layout to the template
template.main.append(layout)

# Define the Panel app
def panel_app():
    return template
# Start the server
rand_port = 5000+random.randint(1,999)
pn.serve({'/': panel_app}, port=rand_port, show=True)





The provided color (SWE Reanalysis) is invalid. Using the default black color.
'#SWE Reanalysis' is not in web format. Need 3 or 6 hex digit.
The provided color (SWE Calculated) is invalid. Using the default black color.
'#SWE Calculated' is not in web format. Need 3 or 6 hex digit.
Launching server at http://localhost:5076


Code received in create_plot: 1401000107
